In [1]:
!git clone https://github.com/16srivarshitha/Transformer_using_pytorch.git

Cloning into 'Transformer_using_pytorch'...
remote: Enumerating objects: 706, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 706 (delta 30), reused 32 (delta 14), pack-reused 655 (from 1)
Receiving objects: 100% (706/706), 127.77 KiB | 2.41 MiB/s, done.
Resolving deltas: 100% (496/496), done.


In [2]:
%cd Transformer_using_pytorch
%pwd

/kaggle/working/Transformer_using_pytorch


'/kaggle/working/Transformer_using_pytorch'

In [3]:
!pip install -r requirements.txt
!pip install --upgrade datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjit

In [4]:
!python src/train_tokenizer.py

Loading 'bentrevett/multi30k' for tokenizer training...
README.md: 1.15kB [00:00, 6.23MB/s]
train.jsonl: 4.60MB [00:00, 68.2MB/s]
val.jsonl: 164kB [00:00, 112MB/s]
test.jsonl: 156kB [00:00, 134MB/s]
Generating train split: 100%|██| 29000/29000 [00:00<00:00, 451125.88 examples/s]
Generating validation split: 100%|█| 1014/1014 [00:00<00:00, 407104.84 examples/
Generating test split: 100%|█████| 1000/1000 [00:00<00:00, 452948.60 examples/s]
Dataset loaded successfully with 29,000 samples
Dataset structure: direct keys
Sample EN: Two young, White males are outside near many bushe...
Sample DE: Zwei junge weiße Männer sind im Freien in der Nähe...
Initializing a new BPE tokenizer...
Training the tokenizer with vocab size 32000. This may take a few minutes...
[00:00:00] Tokenize words                 ██████████████████ 28098    /    28098
[00:00:00] Count pairs                    ██████████████████ 28098    /    28098
[00:00:00] Compute merges                 ██████████████████ 31897    /   

In [5]:
!python train.py

Checking Hugging Face cache directory...
Found existing Multi30k cache at: /root/.cache/huggingface/datasets/bentrevett___multi30k
Deleting Multi30k cache to force fresh download...
Multi30k cache deleted successfully.
Loading tokenizer from /kaggle/working/Transformer_using_pytorch/src/../en-de-tokenizer.json...
Loading 'bentrevett/multi30k' dataset...
Generating train split: 100%|██| 29000/29000 [00:00<00:00, 637589.26 examples/s]
Generating validation split: 100%|█| 1014/1014 [00:00<00:00, 310734.58 examples/
Generating test split: 100%|█████| 1000/1000 [00:00<00:00, 344784.55 examples/s]

--- PERFORMING COMPREHENSIVE LEAKAGE CHECK ---
[Leakage Check] Training set has 28997 unique translation pairs.
 No train/val overlaps detected.
Using 29,000 samples for training and 1,014 for validation.
EvaluationMetrics initialized. BOS: 1, EOS: 2, PAD: 3
Continuing with training...
Starting training...

EPOCH 1/10
Training Epoch:   0%|                                   | 0/907 [00:00<?, ?it/s]

In [6]:
import os
import torch
from datasets import load_dataset
from functools import partial

from transformers import PreTrainedTokenizerFast
from src.dataset import TranslationDataset, collate_fn
from src.transformer import EnhancedTransformer
from src.trainer import Trainer
from config.model_config import ModelConfig
from config.training_config import TrainingConfig

# ==== CONFIG ====
# Path to saved tokenizer and best model
TOKENIZER_PATH = "en-de-tokenizer.json"
BEST_MODEL_PATH = "best_model.pth"
MAX_SEQ_LEN = 512  # should match your training setup
BATCH_SIZE = 64    # should match your TrainingConfig

# ==== DEVICE ====
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ==== LOAD TOKENIZER ====
tokenizer = PreTrainedTokenizerFast(tokenizer_file=TOKENIZER_PATH)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = 3
if tokenizer.bos_token_id is None:
    tokenizer.bos_token_id = 1
if tokenizer.eos_token_id is None:
    tokenizer.eos_token_id = 2

# ==== LOAD CONFIGS ====
model_config = ModelConfig()
training_config = TrainingConfig()
# Ensure consistency with training run
model_config.max_seq_len = MAX_SEQ_LEN
training_config.batch_size = BATCH_SIZE

# ==== BUILD MODEL ====
model_config.vocab_size = tokenizer.vocab_size
model = EnhancedTransformer(model_config, tokenizer).to(device)

# ==== LOAD BEST CHECKPOINT ====
if os.path.exists(BEST_MODEL_PATH):
    # Load full checkpoint including custom classes (only do this if you trust the checkpoint)
    ckpt = torch.load(BEST_MODEL_PATH, map_location=device, weights_only=False)
    model.load_state_dict(ckpt['model_state_dict'])
    print(f"Loaded best model from {BEST_MODEL_PATH} (Perplexity: {ckpt.get('perplexity','N/A')})")
else:
    print(f"Best checkpoint not found at {BEST_MODEL_PATH}, using current weights!")

# ==== BUILD TEST LOADER ====
print("Loading Multi30k test split...")
test_data = load_dataset("bentrevett/multi30k", split="test")
pad_id = tokenizer.pad_token_id
test_dataset = TranslationDataset(list(test_data), tokenizer, MAX_SEQ_LEN)
test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=training_config.batch_size,
    collate_fn=partial(collate_fn, pad_token_id=pad_id),
    shuffle=False
)

# ==== INIT TRAINER ====
trainer = Trainer(model, tokenizer, training_config, device=device)

# ==== EVALUATE ON TEST SET ====
with torch.no_grad():
    test_ppl, test_bleu = trainer.validate(test_loader)

print("="*60)
print(f" Final Test Perplexity: {test_ppl:.4f}")
print(f" Final Test BLEU Score: {test_bleu:.4f}")
print("="*60)

# ==== SHOW SAMPLE TRANSLATIONS ====
from itertools import islice
print("\nSample translations from test set:\n")
model.eval()
for batch in islice(test_loader, 1):  # first batch only
    src = batch['src'].to(device)
    tgt = batch['tgt'].to(device)
    preds, _ = trainer.evaluator.generate_translations(model, [batch], device)
    for i in range(min(5, len(preds))):
        src_text = tokenizer.decode(src[i], skip_special_tokens=True)
        tgt_text = tokenizer.decode(tgt[i], skip_special_tokens=True)
        print(f"SRC: {src_text}")
        print(f"REF: {tgt_text}")
        print(f"PRED: {preds[i]}")
        print("-"*40)


Using device: cuda
Loaded best model from best_model.pth (Perplexity: 9.718684421359837)
Loading Multi30k test split...
EvaluationMetrics initialized. BOS: 1, EOS: 2, PAD: 3

--- Running Validation ---


Generating Translations:   0%|          | 0/16 [00:00<?, ?it/s]

Step 0: logits range -8.549 to 13.966
Top 5 tokens: [117, 161, 149, 1276, 772]
Step 1: logits range -9.820 to 14.079
Top 5 tokens: [149, 136, 7092, 2858, 117]
Step 2: logits range -10.602 to 12.922
Top 5 tokens: [139, 103, 145, 13, 136]


Generating Translations:   6%|▋         | 1/16 [00:01<00:27,  1.84s/it]


DEBUG Sample 0-0:
  Source: A man in an orange hat starring at something .
  Pred: 'Ein Mann mit orangefarbenem Hut starrt auf etwas .'
  Ref:  'Ein Mann mit einem orangefarbenen Hut , der etwas an starrt .'

DEBUG Sample 0-1:
  Source: A Boston Terrier is running on lush green grass in front of a white fence .
  Pred: 'Ein sehr Schlamm - Quarterback , der vor einem weißen Zaun auf einem weißen Zaun rennt .'
  Ref:  'Ein Boston Terrier läuft über saft ig - grünes Gras vor einem weißen Zaun .'
Step 0: logits range -8.161 to 13.875
Top 5 tokens: [117, 336, 161, 723, 772]
Step 1: logits range -9.310 to 16.648
Top 5 tokens: [336, 556, 723, 149, 1504]
Step 2: logits range -10.145 to 14.736
Top 5 tokens: [126, 13, 139, 369, 336]


...


EVALUATION SUMMARY:
Total predictions: 1000
Average prediction length: 12.6 words
Average reference length: 12.8 words

VALIDATION ANALYSIS:
Total predictions: 1000
Prediction Patterns:
  Empty: 0/1000 (0.0%)
  Period-only: 0/1000 (0.0%)
  Single word: 0/1000 (0.0%)
  Multi-word: 1000/1000 (100.0%)
Vocabulary: 1355 unique / 12638 total (ratio: 0.107)

Sample Predictions:
  1. Pred: 'Ein Mann mit orangefarbenem Hut starrt auf etwas .'
     Ref:  'Ein Mann mit einem orangefarbenen Hut , der etwas ...'
  2. Pred: 'Ein sehr Schlamm - Quarterback , der vor einem wei...'
     Ref:  'Ein Boston Terrier läuft über saft ig - grünes Gra...'
  3. Pred: 'Ein Mädchen in Sportkleidung wie man einen Stock m...'
     Ref:  'Ein Mädchen in einem Karateanzug bricht ein Brett ...'
 Final Test Perplexity: 9.2531
 Final Test BLEU Score: 26.9702

Sample translations from test set:



Generating Translations:   0%|          | 0/1 [00:00<?, ?it/s]

Step 0: logits range -8.549 to 13.966
Top 5 tokens: [117, 161, 149, 1276, 772]
Step 1: logits range -9.820 to 14.079
Top 5 tokens: [149, 136, 7092, 2858, 117]
Step 2: logits range -10.602 to 12.922
Top 5 tokens: [139, 103, 145, 13, 136]


Generating Translations: 100%|██████████| 1/1 [00:01<00:00,  1.80s/it]


DEBUG Sample 0-0:
  Source: A man in an orange hat starring at something .
  Pred: 'Ein Mann mit orangefarbenem Hut starrt auf etwas .'
  Ref:  'Ein Mann mit einem orangefarbenen Hut , der etwas an starrt .'

DEBUG Sample 0-1:
  Source: A Boston Terrier is running on lush green grass in front of a white fence .
  Pred: 'Ein sehr Schlamm - Quarterback , der vor einem weißen Zaun auf einem weißen Zaun rennt .'
  Ref:  'Ein Boston Terrier läuft über saft ig - grünes Gras vor einem weißen Zaun .'

EVALUATION SUMMARY:
Total predictions: 64
Average prediction length: 12.4 words
Average reference length: 13.1 words
SRC: A man in an orange hat starring at something .
REF: Ein Mann mit einem orangefarbenen Hut , der etwas an starrt .
PRED: Ein Mann mit orangefarbenem Hut starrt auf etwas .
----------------------------------------
SRC: A Boston Terrier is running on lush green grass in front of a white fence .
REF: Ein Boston Terrier läuft über saft ig - grünes Gras vor einem weißen Zaun .
PRED